# Train Set Pre-processing for Model-1

In [1]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize
import xapian
import string
import sys
import time
import glob
from pandas import DataFrame
import spacy
import import_ipynb
from Features_extraction import *

importing Jupyter notebook from Features_extraction.ipynb


In [2]:
import swifter

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
train_file = 'traing_with_text.json'  
train_set = pd.read_json(open(train_file),orient='index')

In [7]:
start = time.time()
# Open the database for searching.
database = xapian.Database("db-index")
# Start an enquire session.
enquire = xapian.Enquire(database)

In [8]:
doc=[]
def get_topk_evidence(df):
    index = df["index"]
    claim = df["claim"]
    label = df["label"]
    evidence = df["evidence"]
    query_string = claim
    # Parse the query string to produce a Xapian::Query object.
    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    qp.set_database(database)
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)

    # Find the top 10 results for the query.
    enquire.set_query(query)
    matches = enquire.get_mset(0, 5)

    
    
    for m in matches:
        a = m.document.get_data().decode('utf-8')
        tile_pageid = a.split(' ', 2)[:2]
        text = a.split(' ', 2)[2] 
        if str(tile_pageid[1]).isdigit():
            doc.append(dict(
                index = index,
                claim = claim,
                label = label,
                evidence = evidence,
                ir_lookup = a,
                evidence_topk = tile_pageid,
                text_topk =text,
                percentage = m.percent,
                weight = m.weight,            
            ))

In [9]:
nun = train_set.swifter.apply(get_topk_evidence,axis=1)

In [10]:
train_set = pd.DataFrame(doc)

In [ ]:
def get_lenGold(df):
    check = tuple([df['evidence'], df['evidence_topk'])
    if check in set(tuple(i) for i in df['evidence']):
        return 1
    else:
        return 0

In [ ]:
train_set['isGold'] = train_set.apply(get_lenGold, axis=1)

# Feature Extraction

In [ ]:
jaccard_sim = []
lcs_data = []
spacy_sim = []
cosine_sim = []
ngram = []

    
for i, val in train_set.iterrows():
    claim = val['claim']
    evidence = val['text_topk']

    #Jaccard Similarity 
    jacc = jaccardSim(claim, evidence)
    jaccard_sim.append(jacc)
    
    #LCS
    lcs_count = lcs(str(claim), str(evidence))
    lcs_data.append(lcs_count)
    
    #spacy simlarity
    sim = (nlp(claim)).similarity(nlp(evidence))
    spacy_sim.append(sim)
    
    #cosine_sim_tfidf
    document = [claim,evidence]
    cosine_sim.append(cosineSimTfidf(document))
    
    #NGram compare
    ngram_comp = NGram.compare(claim,evidence)
    ngram.append(ngram_comp)
    
    
train_set['jaccard_sim'] = jaccard_sim
train_set['cosine_sim_tfidf'] = cosine_sim
train_set['spaCy_similarity'] = spacy_sim
train_set['lcs'] = lcs_data
train_set['ngram'] = ngram

In [ ]:
train_set.head()

In [18]:
train_set.to_csv('train_set_with_features.csv') #to csv